## bionX Optimimus BMS Reverse Engineer
### Shaun Bowman
### 2022/03/08


#### MCU
#### Freescale S912 Series
#### PN: S912 ZVCA19 MON23N CTQJBJ

[S912 Family Ref Manual NXP / Freescale](https://datasheet.datasheetarchive.com/originals/dk/DKDS42/DSANUWW0048020.pdf#page=22&zoom=100,0,0)
[

#### Notes:
Crazy amounts of Silastic, made opening almost impossible
Appears to be stuck in latched fault
42 V OCV Nom
12S2P Cell Configuration

### Battery Monitorying
#### PN: BQ7694003 (Full label on chip: 83AL8ZT G4 BQ7694003)
[datasheet from alldatasheet](https://www.alldatasheet.com/view.jsp?Searchword=BQ7694003&sField=2)

[direct version](https://pdf1.alldatasheet.com/datasheet-pdf/view/810034/TI1/BQ7694003DBTR.html)

[link to part on digikey](https://www.digikey.com/en/products/detail/texas-instruments/BQ7694003DBTR/6571647)

[GREAT resource comparing various BMS chips -  this one sucks..](https://vasilisks.wordpress.com/2017/12/11/battery-ic-cell-monitor-overview-ltc6803-bq76920-isl94203/)


Whoahhhhhh.... there might very well be a fault set on the battery monitorying chip! for uv... needs to be cleared by host by setting register SYS_STAT (0x00) to 1's - see Table 7-2 in datasheet, page 33. Also there appears to be an override pin....

i2c interface from battery monitor, to host microcontroller
pins:

- SDA 4
- SCL 5
- BAT 20, pwr FOR bat monitor
- REGOUT 8, pwr TO supervisory controller

Alert must be cleared from microcontroller, to battery monitor via i2c

##### Terminology:
- Vss : Ground
- VCC : Logic pwr supply
- BAT : Battery + terminal
- REGOUT,Output LDO : Power supply out, to controller? --> seems to from C39, pin 8 of IC (REGOUT) to gold plated circular via by pin 1 of cpu
- TS1 : Thermistor #1 pos term, pin 6

##### Lithium Ion Battery Referance:
- OCV voltage vs state of charge
:--
![OCVlion](https://www.researchgate.net/publication/324431195/figure/fig12/AS:614090741391382@1523422050880/Li-Ion-battery-OCV-versus-SOC-for-a-UDDS-cycle-charging-current-profile-test.png)

### I2C Bus
#### Pinout BMS chip to header on board, connect to raspberry pi 
[datasheet from alldatasheet](https://www.alldatasheet.com/view.jsp?Searchword=BQ7694003&sField=2)

![I2CPins](https://share.icloud.com/photos/02fle25NmPhlUZQutC2rHBK8g)


#### Register Details
##### SYS_STAT (0x00)

| BIT | 7 | 6 | 5 | 4 | 3 | 2 | 1 | 0 |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| NAME | CC_READY | RSVD | DEVICE_XREADY | OVRD_ALERT | UV | OV | SCD | OCD |
| RESET | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| ACCESS | RW | RW | RW | RW | RW | RW | RW | RW |

- Bits in SYS_STAT may be cleared by writing a "1" to the corresponding bit.
- Writing a "0" does not change the state of the corresponding bit.


https://share.icloud.com/photos/02fle25NmPhlUZQutC2rHBK8g

***
##### CC_READY (Bit 7)
##### : Indicates that a fresh coulomb counter reading is available.
Note that if this bit is not
cleared between two adjacent CC readings becoming available, the bit remains latched to 1. This bit may
only be cleared (and not set) by the host.
0 = Fresh CC reading not yet available or bit is cleared by host microcontroller.
1 = Fresh CC reading is available. Remains latched high until cleared by host.
***
##### RSVD (Bit 6)
##### : Reserved. Do not use.
***
##### DEVICE_XREADY (Bit 5)
##### : Internal chip fault indicator.
When this bit is set to 1, it should be cleared by
the host. May be set due to excessive system transients. This bit may only be cleared (and not set) by
the host.
0 = Device is OK.
1 = Internal chip fault detected, recommend that host microcontroller clear this bit after waiting a
few seconds. Remains latched high until cleared by the host.
***
##### OVRD_ALERT (Bit 4)
##### : External pull-up on the ALERT pin indicator.
Only active when ALERT pin is not
already being driven high by the AFE itself.
0 = No external override detected
1 = External override detected. Remains latched high until cleared by the host.
***
##### UV (Bit 3)
##### : Undervoltage fault event indicator.
0 = No UV fault is detected.
1 = UV fault is detected. Remains latched high until cleared by the host.
***
##### OV (Bit 2)
##### : Overvoltage fault event indicator.
0 = No OV fault is detected.
1 = OV fault is detected. Remains latched high until cleared by the host.